In [76]:
from matplotlib import style
style.use("fivethirtyeight")

In [77]:
import matplotlib.pyplot as plt
import numpy
import pandas
import datetime

# Reflecting into SQLAlchemy ORM

In [78]:
import sqlalchemy
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

In [79]:
engine = create_engine("sqlite:///resources/hawaii.sqlite")

In [80]:
# reflect the database
Base = automap_base()
Base.prepare(autoload_with=engine)

In [81]:
Base.classes.keys()

['station', 'measurement']

In [82]:
# tables
Station = Base.classes.station
Measurement = Base.classes.measurement

In [83]:
# session
session = Session(engine)